In [ ]:
#### WARNING: DOWNLOADED TO AND STARTED WORKING IN ON APP1

In [364]:
import os, json, operator, requests, pandas as pd
from bs4 import BeautifulSoup
#from pandas.io.json import json_normalize

In [365]:
with open('../data/processed/hamilton_data.json', 'r') as f:
    haml = json.load(f)

In [366]:
for key in haml[0]:
    print(key)

act#
track#
lyrics
track


In [367]:
for song in haml:
    song['act#'] = int(song['act#'])
    song['track#'] = int(song['track#'])
haml.sort(key=operator.itemgetter('act#','track#'))
#for song in haml:
#    print('Act:', song['act#'], 'Track#:', song['track#'], 'Track:', song['track'])

In [368]:
basestats = json_normalize(haml)[['act#', 'track#','track','lyrics']]

In [369]:
line_length, word_length = [], []
for song in haml:    
    line_length.append(len(song['lyrics']))
    words_in_song = 0
    for line in song['lyrics']:
        words_in_song += len(line['tokenized'])
    word_length.append(words_in_song)    
basestats['#lines'] = line_length
basestats['#words'] = word_length

In [370]:
r = requests.get('https://en.wikipedia.org/wiki/Hamilton_(album)')

soup = BeautifulSoup(r.content, 'html.parser')

# The Wikipedia page has three tables: The merging suggestion box, the content table on the right, and the main table
# which holds all the countries and capitals. I want the last one:
tbl1 = soup.find_all('table')[2]
tbl2 = soup.find_all('table')[3]
# <table> denotes a table, <tr> a table row, <th> a header, and <td> table content. Get all rows
rows1 = tbl1.find_all('tr')
rows2 = tbl2.find_all('tr')

In [371]:
song_length = []
for row in rows1[:-1]:  # Exclude last line, cause it's just a summary
    if len(row.find_all('td'))==0:
        continue  # Exclude empty rows
    row_compl = row.find_all('td')  # Find all cells
    song_length.append(row_compl[3].text)  # Get row giving the length of song
for row in rows2[:-1]:  # Exclude last line, cause it's just a summary
    if len(row.find_all('td'))==0:
        continue  # Exclude empty rows
    row_compl = row.find_all('td')  # Find all cells
    song_length.append(row_compl[3].text)  # Get row giving the length of song
basestats['len_seconds'] = [int(m[0])*60+int(m[2:4]) for m in song_length]

In [372]:
all_speakers = []
speakers_song_ls = []
for song in haml:
    speakers_song = []
    for line in song['lyrics']:
        all_speakers.append(line['speakers'])
        speakers_song.append(line['speakers'])
    #speakers_song_ls.append(set(speakers_song))
    speakers_song_ls.append(set([item for sublist in speakers_song for item in sublist]))
    #print(set([item for sublist in speakers_song for item in sublist]))
all_speakers = set([item for sublist in all_speakers for item in sublist])
basestats['speakers'] = speakers_song_ls

In [373]:
num_speakers = []
for spkls in speakers_song_ls:
    num_speakers.append(len(spkls))
basestats['#speakers'] = num_speakers


In [379]:
basestats['#words/sec'] = basestats['#words']/basestats['len_seconds']

In [381]:
basestats.drop('lyrics', axis=1, inplace=False).to_csv('../data/processed/basestats.csv', sep=',')

In [380]:
basestats

,act#,track#,track,lyrics,#lines,#words,len_seconds,speakers,#speakers,#words/sec
0,1,1,Alexander Hamilton,"[{'line#': 0, 'tokenized': ['How', 'does', 'a'...",93,615,236,"{ELIZA HAMILTON, HAMILTON, WASHINGTON, JOHN LA...",20,2.605932
1,1,2,"Aaron Burr, Sir","[{'line#': 0, 'tokenized': ['1776', 'New', 'Yo...",66,456,156,"{HAMILTON, MULLIGAN, COMPANY, LAFAYETTE, LAURE...",6,2.923077
2,1,3,My Shot,"[{'line#': 0, 'tokenized': ['I', 'am', 'not', ...",192,1198,333,"{HAMILTON, ENSEMBLE, MULLIGAN, COMPANY, LAFAYE...",7,3.597598
3,1,4,The Story Of Tonight,"[{'line#': 0, 'tokenized': ['I', 'may', 'not',...",32,200,91,"{FULL ENSEMBLE, HAMILTON, ENSEMBLE, MULLIGAN, ...",6,2.197802
4,1,5,The Schuyler Sisters,"[{'line#': 0, 'tokenized': ['There', 's', 'not...",122,613,186,"{SCHUYLER SISTERS, MEN, COMPANY, ALL WOMEN, AL...",13,3.295699
5,1,6,Farmer Refuted,"[{'line#': 0, 'tokenized': ['Hear', 'ye', 'hea...",56,300,112,"{HAMILTON, ENSEMBLE, MULLIGAN, COMPANY, SEABUR...",7,2.678571
6,1,7,You'll Be Back,"[{'line#': 0, 'tokenized': ['You', 'say'], 'no...",44,349,208,"{FULL ENSEMBLE, KING GEORGE}",2,1.677885
7,1,8,Right Hand Man,"[{'line#': 0, 'tokenized': ['British', 'Admira...",180,988,321,"{HAMILTON, WASHINGTON, MULLIGAN, ENSEMBLE, ELI...",15,3.077882
8,1,9,A Winter's Ball,"[{'line#': 0, 'tokenized': ['How', 'does', 'th...",26,137,69,"{LAURENS, ALL MEN, HAMILTON, BURR, FULL COMPANY}",5,1.985507
9,1,10,Helpless,"[{'line#': 0, 'tokenized': ['Hey', 'hey', 'hey...",150,803,249,"{HAMILTON, ALL WOMEN, WOMEN, LAURENS, ELIZA, B...",7,3.224900
